In [23]:
import numpy as np
import pandas as pd
import os
from pathlib import Path

In [2]:
def average_point_cloud(folder, pattern):
    folder = Path(folder)
    files = list(folder.rglob(pattern))
    
    all_clouds = []
    for file in files:
        df = pd.read_csv(file, header=None)
        all_clouds.append(df.to_numpy())

    # Ensure all point clouds have the same number of points
    lengths = [cloud.shape[0] for cloud in all_clouds]
    if len(set(lengths)) != 1:
        raise ValueError(f"Point clouds have different lengths: {lengths}")
    
    stacked = np.stack(all_clouds)  # Shape: (N_files, N_points, 3)
    avg_cloud = np.mean(stacked, axis=0)  # Shape: (N_points, 3)

    return avg_cloud

In [35]:
# Creating ED average point cloud
folder_path = "results" 
avg_endo = average_point_cloud(folder_path, "points_cloud_endo.csv")
avg_epi = average_point_cloud(folder_path, "points_cloud_epi.csv")
pd.DataFrame(avg_endo).to_csv("avg_points_cloud_endo_ED.csv", index=False, header=False)
pd.DataFrame(avg_epi).to_csv("avg_points_cloud_epi_ED.csv", index=False, header=False)


In [38]:
# Save ES points seperated into endo and epi in same structure as ED
input_folder = "ES_pointclouds_merged"
output_folder = "ES_pointclouds"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.startswith("MAD_") and filename.endswith("_merged_points.txt"):
        # Extract participant ID (number after MAD_)
        parts = filename.split("_")
        participant_id = parts[1]

        filepath = os.path.join(input_folder, filename)
        df = pd.read_csv(filepath, header=None, names=["x", "y", "z"])

        # Split into epi and endo
        df_epi = df.iloc[:801]
        df_endo = df.iloc[801:1602]

        # Create participant folder and save
        participant_folder = os.path.join(output_folder, participant_id)
        os.makedirs(participant_folder, exist_ok=True)
        df_epi.to_csv(os.path.join(participant_folder, "points_cloud_epi_ES.csv"), index=False, header=False)
        df_endo.to_csv(os.path.join(participant_folder, "points_cloud_endo_ES.csv"), index=False, header=False)

        #print(f"Processed participant {participant_id}")



In [42]:
def compute_and_save_motion_and_average(patient_id, ed_folder, es_folder, output_folder):
    ed_path = os.path.join(ed_folder, patient_id)
    es_path = os.path.join(es_folder, patient_id)
    out_path = os.path.join(output_folder, patient_id)
    # The average point cloud is hard coded, so change if needed!
    average_ed_epi = pd.read_csv("avg_points_cloud_epi_ED.csv", header=None)
    average_ed_endo = pd.read_csv("avg_points_cloud_endo_ED.csv", header=None)


    if not os.path.isdir(ed_path) or not os.path.isdir(es_path):
        return

    os.makedirs(out_path, exist_ok=True)

    for label in ["epi", "endo"]:
        ed_file = os.path.join(ed_path, f"points_cloud_{label}.csv")
        es_file = os.path.join(es_path, f"points_cloud_{label}_ES.csv")

        if not os.path.exists(ed_file) or not os.path.exists(es_file):
            print(f"Skipping {patient_id} {label}: missing file")
            continue

        ed = pd.read_csv(ed_file, header=None)
        es = pd.read_csv(es_file, header=None)
        average_ed = average_ed_epi if label == "epi" else average_ed_endo
        print(f"Shapes: ed={ed.shape}, es={es.shape}, average_ed={average_ed.shape}")

        # ensures row-wise alignment, instead of on index
        ed = ed.reset_index(drop=True)
        es = es.reset_index(drop=True)
        average_ed = average_ed.reset_index(drop=True)


        motion = ed - es + average_ed

        motion.to_csv(os.path.join(out_path, f"points_cloud_{label}.csv"), index=False, header=False)

        print(f"Saved motion and average for {patient_id} {label}")

In [43]:
# subtract ED-ES and add average ED

ed_folder = "results"
es_folder = "ES_pointclouds"
output_folder = "pc_motion"
os.makedirs(output_folder, exist_ok=True)

for patient_id in os.listdir(ed_folder):
    compute_and_save_motion_and_average(patient_id, ed_folder, es_folder, output_folder)


Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 95 epi
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 95 endo
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 132 epi
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 132 endo
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 92 epi
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 92 endo
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 309 epi
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 309 endo
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 103 epi
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 103 endo
Shapes: ed=(801, 3), es=(801, 3), average_ed=(801, 3)
Saved motion and average for 168 ep

# How to run without overwriting results
* save pc_motion folder outside of this path and rename to results
* then call saxomode-alignment (with height_info from excel)
* then call saxomode-pc
* resulting scores are in PCA_results